## Import Keras and the dataset and Network models

Keras is a top-level library we will use to interface with TensorFlow. Keras will take care of many things that we usually have to do by hand in TF, and it is easier to understand and use.

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

ModuleNotFoundError: No module named 'keras'

## Import TensorFlow and tools to save the network weights

We still need to import TensorFlow. We will be using two built-in functions in TensorFlow to "freeze", or save, our "graph". These are the trained weights that our algorithm will produce. We will save them so we only have to save once!

In [13]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

ModuleNotFoundError: No module named 'tensorflow'

## Load data function

This function takes the common mnist dataset and grabs a "training" and "test" dataset from it. Each dataset consists of x = 28x28 pixel images and y = the numbers they represent.

The pixel values are converted from standard pixel values (0 to 255) to fractions (0 to 1) for easier computations. The number values are put into a "one-hot encoded" array.

Ex: A "one-hot encoded" array for the number "1" will look like this: [0 1 0 0 0 0 0 0 0 0]

In [2]:
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

## Build model function

This is our network! We "stack" layers on top of one another just like lasagna. We enable this in Keras by creating a "Sequential" type model. To add a layer, we call the .add method. This method takes in one of the layer types we imported. We will use two main layers: Convolutional and MaxPooling2D.

Convolutional layers are layers that train a filter to look at the input image.

MaxPooling2D layers simply take the maximum output of a certain number of pixels and remove the rest. This is a good way of "abstracting" the image, and simplifying the output for the next Convolutional Layer to analyze. The output would be similar if you look at the original image through a foggy lense.

Layer 1: Convolutional layer - parameters are standard for image filters
	
	filters:	64 	(neurons in the layer)
	kernel_size:	3	(how many pixels to look at)
	strides:	1	(how many pixels to move once done analyzing)
	padding:	'same'	(automatically pad)
	activation:	'relu'	(negative values set to 0)
	input_shape:	28x28x1	(analyzes 1 28x28 pixel image at a time)

In [1]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, \
            padding='same', activation='relu', \
            input_shape=[28, 28, 1]))
    # 28*28*64
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 14*14*64

    model.add(Conv2D(filters=128, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 14*14*128
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 7*7*128

    model.add(Conv2D(filters=256, kernel_size=3, strides=1, \
            padding='same', activation='relu'))
    # 7*7*256
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 4*4*256

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

## Train function

Keras function .compile takes a model that we passed it (the ie the network we built up above) and compiles it. This gets it ready to train.
	loss:		categorical_crossentropy	(loss function for our network)
	optimizer:	adadelta			(how we will adjust our weights)
	metrics:	'accuracy'			(what we want to measure progress by)

Then we call .fit. This runs our model, with the specified batch size, epochs, and training/test data. It calculates the accuracy after each batch, updates the weights, runs the next batch, and then repeats this process for the number of epochs specified.

In [ ]:
def train(model, x_train, y_train, x_test, y_test):
    model.compile(loss=keras.losses.categorical_crossentropy, \
                  optimizer=keras.optimizers.Adadelta(), \
                  metrics=['accuracy'])

    model.fit(x_train, y_train, \
              batch_size=BATCH_SIZE, \
              epochs=EPOCHS, \
              verbose=1, \
              validation_data=(x_test, y_test))

## Export function

This function saves the weights of our model once it has been trained into a file that TensorFlow can reuse in another program. The catch is that the network layers and parameters in those layers must be the same, otherwise TensorFlow wouldn't know which weights go where.

In [ ]:
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

## Running our program

Now, let's run our program using these functions!
	Our model name will be 'mnist_convnet'
	Our epochs will be 1 (usually this number is between 10 and 50)
	Our batch-size will be 128 (takes 128 images at a time)

In [ ]:
MODEL_NAME = 'mnist_convnet'
EPOCHS = 1
BATCH_SIZE = 128

import os
import os.path as path

# create 'out' directory if not already created
if not path.exists('out'):
    os.mkdir('out')

# load mnist dataset
x_train, y_train, x_test, y_test = load_data()

# build model
model = build_model()

# train model
train(model, x_train, y_train, x_test, y_test)

# export model and save weights
export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")

NameError: name 'path' is not defined